In [1]:
#Import Libraries
import pandas as pd
import numpy as np

In [2]:
#Loading Original Datasets

# Routes Dataset (All US)
original_all_routes = pd.read_json("/Users/Andrew/Documents/CSC 365 Datasets/Original Datasets/openbeta-usa-routes-aug-2020.jsonlines", lines=True)

#Ratings Dataset
original_ratings = pd.read_pickle("/Users/Andrew/Documents/CSC 365 Datasets/Original Datasets/All_Ratings.pkl.zip", compression= "zip") 

In [13]:
# Routes Dataset (All US)

GRADE_CONVERSION = {
    "5.0" : 0,
    "5.1" : 1,
    "5.2" : 1,
    "5.3" : 2,
    "5.4" : 2,
    "5.5" : 3,
    "5.6" : 4,
    "5.7" : 5,
    "5.8" : 6,
    "5.9" : 7,
    "5.10a" : 8,
    "5.10b" : 9,
    "5.10c" : 10,
    "5.10d" : 11,
    "5.11a" : 12,
    "5.11b" : 13,
    "5.11c" : 14,
    "5.11d" : 15,
    "5.12a" : 16,
    "5.12b" : 17,
    "5.12c" : 18,
    "5.12d" : 19,
    "5.13a" : 20,
    "5.13b" : 21,
    "5.13c" : 22,
    "5.13d" : 23,
    "5.14a" : 24,
    "5.14b" : 25,
    "5.14c" : 26,
    "5.14d" : 27,
    "5.15a" : 28,
    "5.15b" : 29,
    "5.15c" : 30,
    "5.15d" : 31,
}


from geopy.geocoders import Nominatim
import sqlalchemy
import time

geolocator = Nominatim(user_agent="geoapiExercises")

edited_all_routes = original_all_routes

print(edited_all_routes.columns)


#Flatten grade into multiple columns
All_Grade_Categories = ['yds']

# Creating New Flattened Dataframe

All_Type_Categories = ['trad', 'sport', 'other']

route_list = []
for index, row in edited_all_routes.iterrows():
    #Create Base Dictionary
    route_dict = {}
    
    #Routename
    route_dict['route_name'] = row['route_name']

    #Flatten Grade
    if 'YDS' in row['grade'].keys():
        for key in GRADE_CONVERSION.keys():
            if key in row['grade']["YDS"]:
                route_dict["yds"] = GRADE_CONVERSION[key]
    else:
        route_dict['yds'] = np.NaN

    #Flatten type
    if "trad" in row['type'].keys():
        route_dict["trad"] = row['type']["trad"]
    else:
        route_dict["trad"] = False

    if "sport" in row['type'].keys():
        route_dict["sport"] = row['type']["sport"]
    else:
        route_dict["sport"] = False

    if 'trad' not in route_dict.keys() and 'sport' not in route_dict.keys():
        route_dict['other'] = True
    else:
        route_dict['other'] = True

    #Route Id
    if len(row['metadata']['mp_route_id']) > 0:
        route_dict['route_id'] = row['metadata']['mp_route_id']
    else:
        continue

    #Location
    if len(row['location']) > 0:
        route_dict['location'] = row['location'][0]
    else:
        route_dict['location'] = np.NaN

    #Description
    if len(row['description']) > 0:
        route_dict['description'] = row['description'][0]
    else:
        route_dict['description'] = np.NaN

    #protection
    if len(row['protection']) > 0:
        route_dict['protection'] = row['protection'][0]
    else:
        route_dict['protection'] = np.NaN

    # route longituide 
    if len(row['metadata']["parent_lnglat"]) > 0:
        route_dict['route_lon'] = str(row['metadata']['parent_lnglat'][0])
        route_dict['route_lat'] = str(row['metadata']['parent_lnglat'][1])
    else:
        continue
    
    route_dict["state_name"] = np.NaN    

    route_list.append(route_dict)


final_routes = pd.DataFrame(route_list).drop_duplicates(subset = ['route_id'])

final_routes.to_csv("/Users/Andrew/Documents/CSC 365 Datasets/Initialize Table Datasets/routesnew.csv", index = False)


Index(['route_name', 'grade', 'safety', 'type', 'fa', 'description',
       'location', 'protection', 'metadata'],
      dtype='object')


In [ ]:
#Ratings Dataset
edited_ratings = original_ratings

print(edited_ratings.shape[0])
rating_list = []

existing_routes = final_routes.route_id.values
print(existing_routes)


success_counter = 0

user_id_dict = {}
user_counter = 0

for index, row in edited_ratings.iterrows():
    rating_dict = {}

    #User_ID
    if row['users'] in user_id_dict.keys():
        rating_dict['user_id'] = user_id_dict[row['users']]
    else:
        user_counter += 1
        rating_dict['user_id'] = int(user_counter)
        user_id_dict[row['users']] = int(user_counter)
    
    #Route Id
    if f"{row['route_id']}" in existing_routes:
        success_counter += 1
        print(f"Success Counter: {success_counter} Route_ID: {row['route_id']}")
        rating_dict['route_id'] = row['route_id']
    else:
        continue

    #Rating
    rating_dict['rating'] = row['ratings']

    rating_list.append(rating_dict)

    #Rating_ID 
    rating_dict['rating_id'] = success_counter


final_ratings = pd.DataFrame(rating_list)

print(final_ratings)
final_ratings.to_csv("./Initialize Table Datasets/ratings.csv", index = False)



In [12]:
#Climbs Dataset
edited_ratings = original_ratings

print(edited_ratings.shape[0])
climbing_list = []

existing_routes = final_routes.route_id.values
# print(existing_routes)


success_counter = 0

user_id_dict = {}
user_counter = 0
for index, row in edited_ratings.iterrows():
    climbing_dict = {}

    #User_ID
    if row['users'] in user_id_dict.keys():
        climbing_dict['user_id'] = user_id_dict[row['users']]
    else:
        user_counter += 1
        climbing_dict['user_id'] = int(user_counter)
        user_id_dict[row['users']] = int(user_counter)
    
    #Route Id
    if f"{row['route_id']}" in existing_routes:
        success_counter += 1
        print(f"Success Counter: {success_counter} Route_ID: {row['route_id']}")
        climbing_dict['route_id'] = row['route_id']
    else:
        continue

    #climb_id
    climbing_dict['climbing_id'] = success_counter

    climbing_dict['frequency'] = np.NaN
    climbing_dict['intensity'] = np.NaN
    climbing_dict['heart_rate'] = np.NaN
    climbing_dict['systolic_pressure'] = np.NaN
    climbing_dict['diastolic_pressure'] = np.NaN

    climbing_list.append(climbing_dict)
    if success_counter > 1000:
        break


final_climbs = pd.DataFrame(climbing_list)

print(final_climbs)
final_climbs.to_csv("/Users/Andrew/Documents/CSC 365 Datasets/Initialize Table Datasets/climbs.csv", index = False)



1893162
Success Counter: 1 Route_ID: 106528796
Success Counter: 2 Route_ID: 106528796
Success Counter: 3 Route_ID: 115405532
Success Counter: 4 Route_ID: 117653356
Success Counter: 5 Route_ID: 117653356
Success Counter: 6 Route_ID: 107821705
Success Counter: 7 Route_ID: 107821705
Success Counter: 8 Route_ID: 107821705
Success Counter: 9 Route_ID: 107821689
Success Counter: 10 Route_ID: 107821689
Success Counter: 11 Route_ID: 107821689
Success Counter: 12 Route_ID: 107821724
Success Counter: 13 Route_ID: 107821724
Success Counter: 14 Route_ID: 107821724
Success Counter: 15 Route_ID: 107836258
Success Counter: 16 Route_ID: 107836258
Success Counter: 17 Route_ID: 107836258
Success Counter: 18 Route_ID: 107821666
Success Counter: 19 Route_ID: 107821666
Success Counter: 20 Route_ID: 107821747
Success Counter: 21 Route_ID: 107821747
Success Counter: 22 Route_ID: 107821747
Success Counter: 23 Route_ID: 107821747
Success Counter: 24 Route_ID: 107821747
Success Counter: 25 Route_ID: 107831729
S

In [94]:
#User Table

user_list = []

for index, row in final_ratings.iterrows():
    user_dict = {}

    user_dict['user_id'] = int(row['user_id'])

    user_dict['name'] = np.NaN
    user_dict['email'] = np.NaN
    user_dict['age'] = np.NaN

    user_list.append(user_dict)

final_users = pd.DataFrame(user_list).drop_duplicates(subset = ['user_id'])

print(final_ratings)
final_users.to_csv("./Initialize Table Datasets/users.csv", index = False)

      user_id   route_id  rating  rating_id
0           1  106528796     3.0          1
1           2  106528796     3.0          2
2           3  115405532     3.0          3
3           4  117653356     3.0          4
4           5  117653356     2.0          5
...       ...        ...     ...        ...
9995     1173  105878191     2.0       9996
9996      902  105878191     2.0       9997
9997     1202  105878191     2.0       9998
9998       46  105878191     2.0       9999
9999      105  105878191     2.0      10000

[10000 rows x 4 columns]
